# Week 10: 2016/03/28-04/01

## Monday

Happy Easter! No reading today since it's a holiday.

## Tuesday class

Now that we've proven that one language ($A_{\mathsf{TM}}$) is undecidable, we can use it to prove that other languages are undecidable.

### The halting problem

The first example is the halting problem. In many textbooks (and the poem "Scooping the Loop Snooper"), the halting problem is actually the prototypical undecidable language, but Sipser does things a little differently.

The proof is not difficult but it has a weird feel to it. The first thing that you have to remember is that the direction of the reduction is the opposite of what most people intuitively think of first. If you want to show that the halting problem is undecidable, you do _not_ reduce the halting problem to $A_{\mathsf{TM}}$. You assume that the halting problem _is_ decidable, then show via a reduction from $A_{\mathsf{TM}}$ to the halting problem that $A_{\mathsf{TM}}$ would also be decidable, which is a contradiction.

Recall that we designed $U$, a universal TM, that _recognizes_ $A_{\mathsf{TM}}$, but we showed that we cannot design a TM $S$ that _decides_ $A_{\mathsf{TM}}$. Practically, the sticking point is that on input $\langle M, w\rangle$, if $M$ loops on $w$, we have no way to detect it so that we can reject. Enter $R$, a hypothetical TM that decides the halting problem $\mathit{HALT}_{\mathsf{TM}}$. If we had such a machine, then designing $S$ would be easy: Use $R$ to check for looping. if a loop is detected, reject; if no loop is detected, we can safely simulate $M$ and return what it returns. But last time we showed (by diagonalization) that $S$ does not exist. Therefore, $R$ cannot exist either.

### Properties of Turing machine runs

Let's take another example not in the chapter; it's Problem 5.12. We say that a Turing machine _erases_ a symbol just when it writes a blank symbol over a nonblank symbol. Show that it is undecidable whether a given Turing machine $M$, when run on input $w$, ever erases a symbol.

First note that it's easy to recognize such situations: just simulate $M$ on $w$ (as in the universal Turing machine) and watch to see whether it erases a symbol. If it does, halt the simulation and accept. If $M$ halts and rejects, reject. If $M$ loops, then loop. But _deciding_ this problem is a whole different story.

Suppose, for the sake of contradiction, that this problem is decidable, that is, there exists a TM $R$ that decides, on input $\langle M, w\rangle$, whether $M$, when run on $w$, ever erases a symbol.

We want to somehow use $R$ to construct a new TM $S$ that decides $A_{\mathsf{TM}}$. If we can do that, we will obtain the desired contradiction (since we know that $A_{\mathsf{TM}}$ is undecidable).

Slightly flawed, but simple attempt: Given any $M$, we can construct a TM $M'$ that, on input $w$:

1. Simulate $M$ on $w$.
2. If $M$ accepts $w$, then write a symbol, erase it, and accept.
3. Otherwise, do nothing and accept.

This TM $M'$ is like an adapter between the two problems at hand. It links the question $S$ wants to decide (does $M$ accept $w$?) with the behavior that $R$ has the magic ability to detect (does $M'$ ever erase?).

Then, define $S$ as follows. On input $\langle M, w\rangle$:

1. Use $M$ to construct $M'$ as described above.
2. Run $R$, the erasure-detector, on $\langle M', w \rangle$.
3. If $R$ detects an erasure, then accept.
4. Otherwise, reject.

If $M$ accepts $w$, then $M'$ would write a symbol and erase it. So $R$ detects an erasure, and so $S$ accepts.

If $M$ rejects $w$, then $M'$ would not erase anything. So $R$ detects no erasure, and so $S$ rejects.

Similarly, if $M$ loops on $w$, then $M'$ would loop, but it also wouldn't erase anything. So $R$ detects no erasure, and so $S$ rejects.

So, $S$ decides $A_{\mathsf{TM}}$. But we know that $A_{\mathsf{TM}}$ is undecidable, which is a contradiction. We conclude that the erasure-detection problem is also undecidable.

The flaw in the above proof is simply that $M$ itself might erase. And if $M'$ uses its own tape to simulate $M$'s tape, as would be natural to do, then there's a potential bug: if $M$ erases a symbol but ultimately rejects, then $M'$, too, will erase a symbol inadvertently. To fix the flaw, simply define a "virtual blank symbol" $b$, and if $M$ ever erases, then $M'$ should write $b$ instead of a real blank.

**Another version, with applications to computer security** A potential source of confusion is that there are so many Turing machines flying around. We could restate this argument using a mix of TMs and Python programs to help keep them separate.

Let's show that it is undecidable whether a given Python program $P$ ever writes to the filesystem.

Suppose, for the sake of contradiction, that this is decidable. That is, there exists a TM $R$ that, on input $\langle P\rangle$, accepts if and only if $P$ would write to the filesystem.

We're going to build another TM, called $S$, that somehow uses $R$ to decide $A_{\mathsf{TM}}$. It goes like this: On input $\langle M, w\rangle$:

1. Translate $M$ into a Python function called `m`.
2. Convert $w$ into a Python string called `w`.
3. Construct the Python program, called $P$:
```
    import os
    if m(w):
        os.system("rm -rf /")
```
4. Run $R$, the write-detector, on $P$.
5. If $R$ detected a write, then _accept_.
6. Otherwise, _reject_.

If $M$ accepts $w$, then $P$ would wipe out your files. $R$ detects this, and so $S$ accepts.

If $M$ rejects $w$, then $P$ does not wipe out your files. $R$ does not detect any writes, and so $S$ rejects.

Similarly, if $M$ loops on $w$, then $P$ would run forever but would not wipe out your files. $R$ does not detect any writes, and so $S$ rejects.

Thus $S$ decides $A_{\mathsf{TM}}$ as desired, which is a contradiction. We conclude that it's undecidable whether a given Python program writes to the filesystem.

Hopefully, it's clear that there's a very wide variety of actions a machine/program can take that are undecidable to detect. (Your homework has one example of an action that _is_ decidable to detect, though.)

### Properties of languages

The next two examples in the book, Theorems 5.2 and 3, are slightly different. They are not questions about a single run of a machine, but about the _entire language_ that the machine recognizes. That is, they are questions of the form "Does $L(M)$ belong to such-and-such a language class?". Theorem 5.2 is about the class $\{\emptyset\}$. Theorem 5.3 is about the class of regular languages.

The basic shape of the proof is still the same. Assume that the question is decidable, so there is a TM $R$ that decides it. As before, we need to construct a new TM $S$ that decides $A_{\mathsf{TM}}$. As before, $S$ has to construct an "adapter" (called $M_1$ or $M_2$ in the book) that links the question that $S$ wants to decide (does $M$ accept $w$?) with the question that $R$ has the magic ability to answer.

In the case of Theorem 5.2, one way to construct the adapter $M_1$ is, on input $x$:

1. If $x \neq w$, _reject_.
2. Otherwise, run $M$ on input $w$ and _accept_ if $M$ does.

Note that, unlike $M'$ in the erasure-detection example, the string $w$ is "baked into" $M_1$.

Let's think carefully about what $M_1$ would do.

If $M$ accepts $w$, then $M_1$ also accepts $w$ but rejects all $x \neq w$. That is, $M_1$ recognizes $\{w\}$, a non-empty language.

If $M$ rejects $w$, then $M_1$ rejects both $w$ and all $x \neq w$. That is, $M_1$ recognizes $\emptyset$.

If $M$ loops on $w$, then $M_1$ also loops on $w$ and rejects all $x \neq w$. So, $M_1$ recognizes $\emptyset$.

Finally, we define $S$ to be the machine that, on input $\langle M, w\rangle$:

1. Construct $M_1$ as described above.
2. Run $R$, the emptiness-detector, on $M_1$.
3. If $R$ detected an empty language, _reject_.
4. Otherwise, _accept_.

If $M$ accepts $w$, then $M_1$ would recognize $\{w\}$, a non-empty language, and so $S$ accepts.

If $M$ rejects or loops on $w$, then $M_1$ would recognize $\emptyset$, and so $S$ rejects.

Thus $S$ decides $A_{\mathsf{TM}}$ as desired, which is a contradiction. We conclude that it's undecidable whether a given Turing machine recognizes the empty language.

The scopes of the variables involved in the above proof can be confusing, so it might or might not help to look at the TMs as Python functions:

In [2]:
def is_empty(function): # This is R
    """Returns True iff function recognizes \emptyset."""
    # Insert magic code here

def accepts(M, w): # This is S
    def M1(x):
        """Recognizes {w} if M(w) is True,
           recognizes \emptyset otherwise."""
        if x != w:
            return False
        else:
            return M(w)
    return not is_empty(M1)

The proof Theorem 5.3 (regularity) is quite similar. We construct $M_2$ that, on input $x$:

1. If $x$ has the form $\mathtt{0}^n\mathtt{1}^n$, _accept_.
2. Otherwise, run $M$ on input $w$ and _accept_ if $M$ does.

Let's think carefully about what $M_2$ does.

If $M$ accepts $w$, then $M_2$ accepts all strings whether or not they are of the form $\mathtt{0}^n\mathtt{1}^n$. That is, it recognizes $\Sigma^\ast$, a regular language.

If $M$ rejects $w$, then $M_2$ accepts just strings of the form $\mathtt{0}^n\mathtt{1}^n$ and rejects all others. That is, it recognizes $\{\mathtt{0}^n \mathtt{1}^n \mid n \geq 0\}$, a non-regular language.

If $M$ loops on $w$, then $M_2$ accepts just strings of the form $\mathtt{0}^n\mathtt{1}^n$ and loops on all others. That is, it recognizes $\{\mathtt{0}^n \mathtt{1}^n \mid n \geq 0\}$, a non-regular language.

The rest of the proof is very similar to the last one. We define $S$ to be the machine that, on input $\langle M, w\rangle$:

1. Construct $M_2$ as described above.
2. Run $R$, the regularity-detector, on $M_2$.
3. If $R$ detected a regular language, _accept_.
4. Otherwise, _reject_.

If $M$ accepts $w$, then $M_2$ would recognize $\Sigma^\ast$, a regular language, and so $S$ accepts.

If $M$ rejects or loops on $w$, then $M_2$ would recognize $\{\mathtt{0}^n \mathtt{1}^n \mid n \geq 0\}$, a non-regular language, and so $S$ rejects.

Thus $S$ decides $A_{\mathsf{TM}}$ as desired, which is a contradiction. We conclude that it's undecidable whether a given Turing machine recognizes a regular language.

It turns out that there is a general recipe for proving results of this type. This is called Rice's Theorem and is the focus of one of the homework questions.

**Question.** Try proving: It is undecidable whether two given Turing machines $M_1$ and $M_2$ recognize the same language.

### Other undecidable problems

Alas, we don't have time to look at more examples of undecidable problems. The examples we've looked at so far may seem a little arcane because they are all questions about Turing machines, but some other examples are:

- Do two context-free grammars generate the same language (Exercise 5.1)?
- The third and hardest puzzle we did on the first day of class (Section 5.2).
- Does a polynomial equation in several variables have an integral solution?

See [the survey by Poonen](http://www-math.mit.edu/~poonen/papers/sampler.pdf) for more examples.

## Wednesday reading

Catch up by reading Section 5.1, up to but not including "Reductions via Computation Histories." 

If you were interested in the third, hardest puzzle that we did on the first day of class, then read the rest of Section 5.1 and Section 5.2, but this is optional.

## Thursday class

Today we'll discuss a question raised on the first day of class: am I a computer? Most people said no, but now we are armed with a formal definition of what a computer is. So the question becomes: Can my mind/brain be simulated by a Turing machine? This is a thorny question to say the least, but one that's worth at least discussing.

There aren't any notes for this topic save for one "no" argument that is technical. Turing called it "the mathematical objection" and it is usually known today as the Penrose-Lucas argument. This version is due to Penrose and comes from [an article criticizing him](http://www.ihmc.us/users/phayes/pub/lafortehayesford.pdf). 

Let $R$ be a decider Turing machine that detects _some_ cases of looping. That is, on input $\langle M, w\rangle$, if $R$ accepts, then $M$ loops on input $w$. But if $R$ rejects, it doesn't mean anything.

Now we go through the usual diagonalization argument; the only difference is that it doesn't lead to a contradiction, but to a machine/input pair that is beyond $R$'s detection abilities. Assume an ordering $M_1, M_2, \ldots$ on Turing machines and an ordering $w_1, w_2, \ldots$ on strings. We can build a big table with the results of $R$ on all machines and inputs:

| | $\varepsilon$ | $\mathtt{0}$ |$\mathtt{1}$ | $\mathtt{00}$ | $\cdots$ |
|-|-------------|------------|-------------|-------------|--------|
|$M_1$| _don't know_ | _loops_ | _don't know_ | _loops_    |        |
|$M_2$| _don't know_ | _loops_ | _loops_ | _don't know_    |        |
|$M_3$| _loops_ | _don't know_ | _loops_ | _don't know_ |  |
|$M_4$| _don't know_ | _don't know_ | _don't know_ | _loops_ |                |
|$\vdots$| | | | | | |

We define $D$ to be the Turing machine that _does_ the opposite of the diagonal of this table. That is, on input $w$:

1. Find $i$ such that $w = w_i$.
2. Run $R$, the partial looping-detector, on $\langle M_i, w_i\rangle$.
3. If $R$ detected an infinite loop, _accept_.
4. Otherwise, go into an infinite loop.

Now, there must be an $i$ such that $D = M_i$. We claim that $\langle D, w_i\rangle$ is an example of a looping machine/input pair that is beyond $R$'s detection abilities. Suppose that $R$, on input $\langle D, w_i\rangle$, detects looping. Then, in fact, $D$ does not loop, which is a contradiction. So $R$ must not detect looping. Then, in fact, $D$ does loop, but $R$ does not detect it.

It's critical that you understand the argument thus far before moving on.

Suppose that you are equivalent to a Turing machine. It should then be possible to construct a Turing machine $Y$ that accepts $\langle M, w\rangle$ in exactly those cases when you are able to detect that $M$ loops on input $w$.

Then, by the above argument, there is a machine/input pair $\langle D, w_i\rangle$ that loops, but $Y$ rejects it.  By the definition of $Y$, you are not able to detect that $\langle D, w_i\rangle$ loops. But if you understood the above argument, then you know that $\langle D, w_i\rangle$ does loop. This is a contradiction! Therefore, you are not equivalent to a Turing machine.

There are many possible responses to this argument -- what do you think?